In [ ]:
import sys
sys.path.append('/home/super/mironov/mia/classifiers')

import torch
from models.resnet import ResNet

resnet = ResNet('classic', 152)
resnet_ckp = torch.load('/home/super/mironov/mia/model_transferring/loaded_models/resnet152.pth', weights_only=False)

def init_model(model: torch.nn.Module):
    for module in model.modules():
        if (isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear)):
            torch.nn.init.kaiming_uniform_(module.weight)

resnet.apply(init_model)
print()

In [ ]:
i = 0
for key, value in resnet.state_dict().items():
    print(f"{i}: {key}: {value.shape}")
    i += 1
print('------------------------------------------------------------------')
i = 0
for key, value in resnet_ckp.items():
    print(f"{i}: {key}: {value.shape}")
    i += 1

In [14]:
mapping = {}
with open('/home/super/mironov/mia/model_transferring/resnet152_weights_mapping.txt', 'r') as f:
    for line in f.readlines():
        cline = line[:-1]
        current_layer = int(cline.split(':')[0])
        
        if ('stem.stem.0.bias' not in cline) and ('num_batches_tracked' not in cline):
            old_layer = int(cline.split('(')[-1].split(')')[0])
        else:
            old_layer = None
            
        mapping[current_layer] = old_layer

i = 0
current_state = {}
for key, value in resnet.state_dict().items():
    current_state[i] = (key, value)
    i += 1
    
i = 0
old_state = {}
for key, value in resnet_ckp.items():
    old_state[i] = value
    i += 1
    
new_state = {}
for i in range(len(current_state)):
    old_i = mapping[i]
    
    if old_i is None:
        value = current_state[i][1]
    else:
        value = old_state[old_i]
        
    new_state[current_state[i][0]] = value

In [ ]:
resnet.load_state_dict(new_state)
torch.save(resnet.state_dict(), '/mnt/disk2/mironov/models/resnet/resnet152_imagenet.pth')

In [ ]:
from models.resnet import ResNetClassifier

classifier = ResNetClassifier('classic', 152, 101)
classifier.apply(init_model)

classifier.resnet.load_state_dict(new_state)

torch.save(classifier.state_dict(), '/home/super/mironov/mia/logs/resnet152_food101.pth')